In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [2]:
num_classes = 7
batch_size = 256
epochs = 5

In [3]:
with open('fer2013.csv') as f:
    content = f.readlines()

lines = np.array(content)

In [4]:
num_of_instances = lines.size
print('number of instances :', num_of_instances)

number of instances : 35888


In [5]:
import pandas as pd

In [6]:
dataset = pd.read_csv('fer2013.csv')

In [7]:
dataset.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [5]:
x_train, y_train, x_test, y_test = [], [], [], []

In [6]:
for i in range(1, num_of_instances):
    try:
        emotion, img, usage = lines[i].split(",")
        val = img.split(" ")
        
        pixels = np.array(val, 'float32')
        
        emotion = keras.utils.to_categorical(emotion,num_classes)
        
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
    except:
        print("", end="")

In [7]:
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train,'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

In [8]:
x_train /= 255
x_test /=255

In [9]:
x_train = x_train.reshape(x_train.shape[0], 48,48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48,48,1)
x_test = x_test.astype('float32')

In [10]:
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

28709 train samples
3589 test samples


In [12]:
model = Sequential()

model.add(Conv2D(64, (5,5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size= (5,5), strides=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2,2)))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2,2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))


In [13]:
gen = ImageDataGenerator()

train_generator = gen.flow(x_train, y_train, batch_size=batch_size)

In [14]:
model.compile(loss='categorical_crossentropy',
             optimizer = keras.optimizers.Adam(),
             metrics = ['accuracy'])

In [ ]:
model.fit_generator( train_generator, steps_per_epoch=batch_size, epochs = epochs)

##saving model weights and structure
model_json = model.to_json()
with open('facial_expression_model_structure.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('facial_expression_model_weights.h5')

print('model saved to disk')

Epoch 1/5
  4/256 [..............................] - ETA: 1:05:45 - loss: 1.9602 - acc: 0.2100